In [27]:
"""
Returns a column for the movement of every point, given the forces matrix.
If no break occurs then there will be only one motion returned
"""
function get_motions(forces::Matrix, strength)
    motion = similar(forces)
    
    for dim in 1:size(forces,1)
        is_north = forces[dim,:].>0
        north_force = sum(forces[dim,is_north])
        south_force = -sum(forces[dim,!is_north])
        @assert(north_force>=0)
        @assert(south_force>=0)
        
        tension = 2*min(north_force, south_force) #This is the resisted force. It that can break, rather than move. 
        if tension>strength
            #breaks, so north forces will go one way, and south the other
            motion[dim,is_north]=(north_force-strength) #/sum(is_north)
            motion[dim,!is_north]=(strength-south_force)  #/sum(!is_north)
        else
            #does not break, so all forces apply same motion
            motion[dim,:]=(north_force-south_force)     #/size(forces,1)
        end
    end
    #Find unique motion rows.
    #These corespond to all the new points
    unique(motion,2)
end

get_motions (generic function with 1 method)

In [35]:
function move!(wordsense_embeddings::Matrix,
    wordsense_col::Int,
               motions::Matrix)
    cur_pos=wordsense_embeddings[:,wordsense_col]
    @show motions
    @show cur_pos
    new_points = motions .+ cur_pos
    #PREM-OPT: The code below works for any number of new points. But if there is only one new point (i.e. no split occured), then we can just replacer the current single point
    other_senses = wordsense_embeddings[:,[1:wordsense_col-1;wordsense_col+1:end]]
    wordsense_embeddings = [other_senses new_points]
end

move! (generic function with 1 method)

In [36]:
word_embeddings = Dict{AbstractString,Matrix}()
word_embeddings["Hi"]=[1. 0; 0 1; 2 3]' #One column per word sense
fs = [1. 2; -1 -2; -0.5 -1; -0.5 0]'
ms=get_motions(fs,2)
move!(word_embeddings["Hi"],2,ms)

motions = [-1.0 -1.0
 0.0 -1.0]

2x4 Array{Float64,2}:
 1.0  2.0  -1.0  -1.0
 0.0  3.0   1.0   0.0


cur_pos = [0.0,1.0]
